Filesystem, 

could either be a directory or a file
directory can have other directory and files

In [1]:
class Node:
    def __init__(self, name):
        self.name = name 


class directory(Node):
    def __init__(self,name):
        super().__init__(name)
        self.children = {}  # {} # can have directories and files
    
    
class file(Node):
    def __init__(self,name,content = ""):
        super().__init__(name)
        self.content = content  # {}
    
    

class FileSystem():
    def __init__(self):
        self.root = directory("")
    
    
    def mkdir(self,path):
        dir_names = [i for i in path.split("/") if i]
        node = self.root 
        
        for each in dir_names:
            if each not in node.children:
                node.children[each] = directory(each)
            node = node.children[each]
        
        return 

    def addContentToFile(self,path,content):
        names = [i for i in path.split("/") if i]
        dir_names = names[:-1]
        file_name = names[-1]
        node = self.root
        for each in dir_names:
            if each not in node.children:
                raise Exception("Directory {each} does not Exist for the following Directories: {dir_names}")
            node = node.children[each]
        File = file(file_name,content)
        node.children[file_name] = File
        return 
    
    def ls(self,path):
        names = [i for i in path.split("/") if i]
        node = self.root
        for each in names:
            if each not in node.children:
                raise Exception("Directory {each} does not Exist for the following Directories: {names}")
            node = node.children[each]
        
        if isinstance(node,file):
            return node.name 

        return sorted(node.children.keys())
        
        
        

In [ ]:
# file system
fs = FileSystem()

fs.ls("/") 
# Returns []

fs.mkdir("/a/b/c")
fs.addContentToFile("/a/b/c/d", "hello")

fs.ls("/") 
# Returns ["a"]

fs.ls("/a/b/c")
# Returns ["d"] -> "d" is the file inside /a/b/c

fs.readContentFromFile("/a/b/c/d")
# Returns "hello"

Problem Statement:We need to design a Bigram Frequency Analyzer.You will receive a stream of sentences. Your system must parse these sentences and track how often pairs of consecutive words (bigrams) appear.Core Requirements:Ingest data: Process sentences one by one.Query data: Return the frequency of a specific bigram (pair of words).Example:Input: "The quick brown fox jumps. The quick brown fox."Bigrams for first sentence: (the, quick), (quick, brown), (brown, fox), (fox, jumps)Query ("quick", "brown") $\to$ Returns 2.Query ("fox", "jumps") $\to$ Returns 1.Constraints:Case-insensitive ("The" == "the").Ignore punctuation ("fox." == "fox").Handle whitespace properly.


Ingest:
O(N) per sentence

Query:
O(1)

Top-K:
O(M log K)
M = number of unique words following given word


issue : Each call scans all followers → O(M log K), what if it is called frequently? : Maintain a heap per word (updated on ingest)

In [ ]:
from collections import defaultdict
class BigRam():
    def __init__(self):
        self.counts = defaultdict(int)
        # self.counts = defaultdict(lambda : defaultdict(int))
    
    
    def ingest(self,data):
        word = data.lower().strip().split()
        for i in range(1,len(word)):
            self.counts[(word[i-1],word[i])] += 1
            # self.counts[word[i-1]][word[i]] += 1
        return
    
    def query(self,word1,word2): # simple query
        return self.counts[(word1,word2)]

    
    def get_top_k_following(self, word: str, k: int) -> List[str]:
        res = []
        
        for chr,count in self.counts[word].items():
            heapq.heappush(res, (count,chr))
            
            if len(res) > k:
                heapq.heappop(res)
        
        return [char for _,char in res]


📅 Round 1: Design a Meeting Scheduler
Problem Statement We want to design a backend for a meeting scheduler system (like a simplified Google Calendar).

Core Requirements

Users: The system has multiple users.

Scheduling: A user can attempt to book a meeting with a list of other users for a specific time interval (start_time, end_time).

Validation: A meeting should only be booked if all participants are free during that interval. If anyone has an overlap, the booking fails.

Visibility: A user can see their own schedule (list of meetings).

Constraints

Time is represented as simple Integers for now (e.g., 0 to 2400 or Unix timestamps) to keep it simple.

Everything is in-memory.

Example Scenario

User A has a meeting [10, 12].

User B is free.

A tries to book a meeting with B from [11, 13].

Result: Failure (A is busy until 12).

A tries to book with B from [13, 14].

Result: Success.

Phase 1: Class Design
Before writing full logic, outline your Classes and Method Signatures.

How will you store a User's schedule?

What methods will the main Scheduler class have?

(Sketch this out in Python-like pseudocode or just text.)

In [ ]:
# schedular will have a map
# the map will map the id of the user to the meetingTimes that the user has
# we can either have double linked list for meeting times or we can have like buckets of half an hour
# for doubly linked list, we can binary search to get the first meeting which is less than the meeting that we are about to schedule
# and then check if the meeting is possible for that user, 


# bookMeeting(self, initiaterId, otherUsersID, interval):
# will do binary search in each otherUsersID(and initiaterID) time to check if the meeting is possible
# Only if the meeting is possible, since its a doubly linked list, we add in the index we found
# otherwise return false

class Schedular():
    def __init__(self):
        self.map = defaultdict(list)
    
    
    def bookMeeting(self, initiaterId, otherUserIds, interval):
        
        allUsers = otherUserIds + [initiaterId]
        n = len(allUsers)
        index = [-1] * n 
        
        for count,each in enumerate(allUsers):
            i = self.binarySearch(each, interval)
            if i == -1:
                return False 
            index[count] = i 
        
        for count,each in enumerate(allUsers):
            self.map[each].insert(index[count], interval)
        
        return True 
        
    
    
    def binarySearch(self, id, interval):
        # returns the index where the interval can go in the id's meetings, -1 if its not possible 
        # array is sorted based on the start time, binary search on start time
        arr = self.map[id]
        potential = len(arr)
        target = interval[0]
        r = len(arr)-1
        l = 0
        
        while l<=r:
            mid = (l+r) // 2
            if arr[mid][0] >= target:
                potential = mid 
                r = mid - 1
            else:
                l = mid + 1
        new_start, new_end = interval
        # 2. Check for overlap with the NEXT meeting
        # If 'potential' is not at the end of the list, we look at arr[potential].
        # Its start time must be >= our new meeting's end time.
        if potential < len(arr):
            next_meeting_start = arr[potential][0]
            if next_meeting_start < new_end:
                return -1

        # 3. Check for overlap with the PREVIOUS meeting
        # If 'potential' is > 0, there is a meeting before this slot.
        # Its end time must be <= our new meeting's start time.
        if potential > 0:
            prev_meeting_end = arr[potential - 1][1]
            if prev_meeting_end > new_start:
                return -1
        
        return potential
    
    def printMeeting(self,userId):
        
        arr = self.map[userId]
        print(f"Meeting times for User : {userId}")
        for start,end in arr:
            print(f"{start} - {end}")
        
        return 
            
        
                
    
    
    


In [ ]:
# we are doing binary search on an array, which is fine, but adding into the array is going to be O(N)
# I'd rather use a binary search tree



class Node:
    def __init__(self, start, end):
        self.start = start 
        self.end = end 
        self.left = None 
        self.right = None 

class Scheduler:
    def __init__(self):
        # Maps user_id -> Root Node of their BST
        self.map = {} 

    def bookMeeting(self, initiaterId, otherUserIds, interval):
        allUsers = otherUserIds + [initiaterId]
        
        # 1. VALIDATION PHASE: Check if everyone is free
        # We traverse everyone's tree without modifying anything yet.
        for user_id in allUsers:
            root = self.map.get(user_id) # Returns None if user has no meetings
            if not self.checkAvailability(root, interval):
                return False 
        
        # 2. INSERTION PHASE: Everyone is free, so book it.
        for user_id in allUsers:
            root = self.map.get(user_id)
            # Update the root in the map (in case the tree was empty and we created a new root)
            self.map[user_id] = self.insert(root, interval)
            
        return True

    def checkAvailability(self, node, interval):
        # Returns True if the interval can be booked, False if there is a conflict
        if not node:
            return True 
        
        new_start, new_end = interval
        
        # LOGIC:
        # 1. If we start after current node ends -> Go Right
        if new_start >= node.end:
            return self.checkAvailability(node.right, interval)
            
        # 2. If we end before current node starts -> Go Left
        elif new_end <= node.start:
            return self.checkAvailability(node.left, interval)
            
        # 3. Otherwise, we are overlapping
        else:
            return False 

    def insert(self, node, interval):
        # Standard BST Insertion
        if not node:
            return Node(interval[0], interval[1])
        
        new_start = interval[0]
        
        if new_start >= node.end:
            node.right = self.insert(node.right, interval)
        else:
            node.left = self.insert(node.left, interval)
            
        return node

    def printInOrder(self, node):
        if not node: return
        self.printInOrder(node.left)
        print(f"[{node.start}-{node.end}]", end=" ")
        self.printInOrder(node.right)

# --- Test Driver ---
s = Scheduler()

# Book for User 1 and 2 at 10-12
print("Book 10-12 (User 1,2):", s.bookMeeting(1, [2], [10, 12]))

# Book for User 1 at 13-14 (Should work, goes to Right of 10-12)
print("Book 13-14 (User 1):  ", s.bookMeeting(1, [], [13, 14]))

# Book for User 1 at 8-9 (Should work, goes to Left of 10-12)
print("Book 08-09 (User 1):  ", s.bookMeeting(1, [], [8, 9]))

# Book for User 1 at 11-13 (Overlap with 10-12) -> Should Fail
print("Book 11-13 (User 1):  ", s.bookMeeting(1, [], [11, 13]))

print("\nUser 1's Calendar:")
s.printInOrder(s.map[1])